# 1. Preprocessing


In [101]:
# # import cv2
# # import numpy as np
# # from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
# # from skimage.morphology import skeletonize
# # from skimage.util import img_as_ubyte
# # from scipy.ndimage import gaussian_filter
# # import math

# # def preprocess_image(image):
# #     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# #     blurred = gaussian_filter(gray, sigma=1.0)
# #     enhanced = cv2.equalizeHist(blurred)
# #     return enhanced


# import cv2
# import numpy as np
# from scipy.ndimage import gaussian_filter
# from skimage.morphology import skeletonize

# def preprocess_image(image):
#     # grayscale -> blur -> normalize -> equalize
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blurred = gaussian_filter(gray, sigma=1.0)
#     # normalize to uint8 for equalizeHist
#     norm = cv2.normalize(blurred, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
#     enhanced = cv2.equalizeHist(norm)
#     return enhanced


# 2. Custom Edge Detection (Gradient + Adaptive Threshold)

In [102]:
# # def custom_edge_detection(image):
# #     # Custom gradient filters
# #     kernel_x = np.array([[-1, 0, 1],
# #                          [-2, 0, 2],
# #                          [-1, 0, 1]])
# #     kernel_y = np.array([[-1, -2, -1],
# #                          [ 0,  0,  0],
# #                          [ 1,  2,  1]])

# #     grad_x = cv2.filter2D(image, -1, kernel_x)
# #     grad_y = cv2.filter2D(image, -1, kernel_y)

# #     magnitude = np.sqrt(grad_x**2 + grad_y**2)
# #     magnitude = np.uint8(magnitude / magnitude.max() * 255)

# #     # Adaptive thresholding
# #     _, edge_map = cv2.threshold(magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
# #     return edge_map

# def custom_edge_map(gray):
#     # Sobel-like kernels, then magnitude and Otsu threshold
#     kx = np.array([[-1,0,1],[-2,0,2],[-1,0,1]], dtype=np.float32)
#     ky = np.array([[-1,-2,-1],[0,0,0],[1,2,1]], dtype=np.float32)
#     gx = cv2.filter2D(gray.astype(np.float32), cv2.CV_32F, kx)
#     gy = cv2.filter2D(gray.astype(np.float32), cv2.CV_32F, ky)
#     mag = np.sqrt(gx**2 + gy**2)
#     mag_u8 = ((mag / (mag.max() + 1e-12)) * 255).astype(np.uint8)
#     _, edge_binary = cv2.threshold(mag_u8, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#     return edge_binary, mag_u8

# 3. Leaf Segmentation

In [103]:
# # def segment_leaf(image):
# #     _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
# #     contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# #     if not contours:
# #         return None, None
# #     largest_contour = max(contours, key=cv2.contourArea)
# #     mask = np.zeros_like(image)
# #     cv2.drawContours(mask, [largest_contour], -1, 255, -1)
# #     return mask, largest_contour

# def segment_largest_object(gray):
#     # Otsu threshold then largest contour
#     _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#     contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     if not contours:
#         return None, None, th
#     largest = max(contours, key=cv2.contourArea)
#     mask = np.zeros_like(gray, dtype=np.uint8)
#     cv2.drawContours(mask, [largest], -1, 255, -1)
#     return mask, largest, th

# 4. Feature Extraction

A. Shape Features

In [104]:
# # def extract_shape_features(contour):
# #     x, y, w, h = cv2.boundingRect(contour)
# #     aspect_ratio = float(w) / h
# #     area = cv2.contourArea(contour)
# #     rect_area = w * h
# #     extent = float(area) / rect_area
# #     hull = cv2.convexHull(contour)
# #     hull_area = cv2.contourArea(hull)
# #     solidity = float(area) / hull_area
# #     moments = cv2.moments(contour)
# #     hu_moments = cv2.HuMoments(moments).flatten()
# #     return [aspect_ratio, extent, solidity] + hu_moments.tolist()

# def compute_entropy(values):
#     if values.size == 0:
#         return 0.0
#     hist, _ = np.histogram(values, bins=256, range=(0,255), density=True)
#     hist = hist + 1e-12
#     return -np.sum(hist * np.log2(hist))

B. Texture Features

In [105]:
# def extract_texture_features(image, mask):
#     masked = cv2.bitwise_and(image, image, mask=mask)
#     lbp = local_binary_pattern(masked, P=8, R=1, method='uniform')
#     lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), density=True)

#     glcm = graycomatrix(masked, [1], [0], levels=256, symmetric=True, normed=True)
#     contrast = graycoprops(glcm, 'contrast')[0, 0]
#     homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
#     entropy = -np.sum(glcm * np.log2(glcm + 1e-10))

#     return lbp_hist.tolist() + [contrast, homogeneity, entropy]

# def extract_vein_density(mask):
#     if mask.sum() == 0:
#         return 0.0, 0
#     sk = skeletonize(mask > 0)
#     sk_count = np.count_nonzero(sk)
#     mask_count = np.count_nonzero(mask)
#     vein_density = float(sk_count) / (mask_count + 1e-12)
#     return vein_density, sk_count

C. Vein Features

In [106]:
# def extract_vein_features(mask):
#     skeleton = skeletonize(mask // 255)
#     vein_density = np.sum(skeleton) / np.sum(mask // 255)

#     # Fractal dimension (box-counting method)
#     def fractal_dimension(Z):
#         def boxcount(Z, k):
#             S = np.add.reduceat(
#                 np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
#                                    np.arange(0, Z.shape[1], k), axis=1)
#             return len(np.where(S > 0)[0])
#         Z = Z < 0.5
#         p = min(Z.shape)
#         n = 2**np.floor(np.log2(p))
#         sizes = 2**np.arange(int(np.log2(n)), 1, -1)
#         counts = [boxcount(Z, size) for size in sizes]
#         coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
#         return -coeffs[0]

#     fractal_dim = fractal_dimension(skeleton)
#     return [vein_density, fractal_dim]

D. Edge Features

In [107]:
# def extract_edge_features(edge_map, mask):
#     edge_masked = cv2.bitwise_and(edge_map, edge_map, mask=mask)
#     edge_density = np.sum(edge_masked) / np.sum(mask)

#     # Orientation histogram
#     sobelx = cv2.Sobel(edge_masked, cv2.CV_64F, 1, 0, ksize=5)
#     sobely = cv2.Sobel(edge_masked, cv2.CV_64F, 0, 1, ksize=5)
#     angles = np.arctan2(sobely, sobelx) * 180 / np.pi
#     hist, _ = np.histogram(angles, bins=8, range=(-180, 180), density=True)

#     return [edge_density] + hist.tolist()

# 5. Feature Fusion

In [108]:
# # def extract_all_features(image):
# #     preprocessed = preprocess_image(image)
# #     edge_map = custom_edge_detection(preprocessed)
# #     mask, contour = segment_leaf(preprocessed)

# #     if mask is None or contour is None:
# #         return None  # Not a leaf

# #     shape_feats = extract_shape_features(contour)
# #     texture_feats = extract_texture_features(preprocessed, mask)
# #     vein_feats = extract_vein_features(mask)
# #     edge_feats = extract_edge_features(edge_map, mask)

# #     feature_vector = shape_feats + texture_feats + vein_feats + edge_feats
# #     return feature_vector

# def is_leaf(image):
#     preprocessed = preprocess_image(image)
#     edge_map = custom_edge_detection(preprocessed)
#     mask, contour = segment_leaf(preprocessed)

#     # If no contour found → Not a leaf
#     if mask is None or contour is None:
#         return False, None  

#     # Extract features
#     shape_feats = extract_shape_features(contour)
#     texture_feats = extract_texture_features(preprocessed, mask)
#     vein_feats = extract_vein_features(mask)
#     edge_feats = extract_edge_features(edge_map, mask)

#     # Unpack important values
#     contour_area = cv2.contourArea(contour)
#     image_area = image.shape[0] * image.shape[1]
#     area_ratio = contour_area / image_area

#     vein_density = vein_feats[0]
#     edge_density = edge_feats[0]

#     # 🔴 Reject conditions (tweak thresholds as needed)
#     if area_ratio > 0.95:   # Contour covers ~whole image → likely plain background
#         return False, None
#     if edge_density < 0.01: # Very few edges → not a leaf
#         return False, None
#     if vein_density < 0.005: # No visible veins → not a leaf
#         return False, None

#     # ✅ Otherwise → Leaf
#     feature_vector = shape_feats + texture_feats + vein_feats + edge_feats
#     return True, feature_vector



In [109]:
# # image = cv2.imread('pngtree-green-leaf.png')

# # features = extract_all_features(image)
# # print("Feature vector shape:", np.shape(features))
# # print("Sample features:", features[:5])  # Show first few values

# print("Contains NaNs:", np.isnan(features).any())
# # print("Contains Infs:", np.isinf(features).any())

# image = cv2.imread("a.jpg")
# is_leaf_flag, features = is_leaf(image)

# if is_leaf_flag:
#     print("✅ This is a LEAF.")
#     print("Feature vector length:", len(features))
# else:
#     print("❌ This is NOT a leaf.")



In [ ]:
# import cv2
# import numpy as np

# img = cv2.imread("a.jpg")
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# variance = np.var(gray)

# if variance < 50:  # threshold, tune it
#     print("Non-leaf (plain green background)")
# else:
#     print("Leaf")



In [6]:
    import cv2
    import numpy as np
    from skimage.morphology import skeletonize
    from skimage.measure import shannon_entropy
    from skimage import img_as_ubyte
    import math

    def compute_entropy(gray_region):
        if gray_region.size == 0:
            return 0.0
        return shannon_entropy(gray_region)

    def segment_largest_object(image, gray):
        """
        Segment largest object.  
        Otsu threshold is primary, HSV mask only if Otsu is too weak.
        """
        
        
        # Otsu
        # Step 1: Otsu thresholding (separates object from background)
        #cv2.THRESH_OTSU-> Automatically select best theresold value
        _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        
        # Step 2: Morphological cleanup (remove noise, fill gaps)
        #MORPH_ELLIPSE → elliptical kernel (better for natural shapes like leaves, circles)
        
        kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
        kernel_open  = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
        cleaned = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel_close) # FILL THE HOLE INSIDE THE LEAF (BLACK)
        cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel_open) #REMOVE TINY NOISE OUTSIDE THE LEAF (WHITE)
        
        
        # Step 3: If Otsu fails → use HSV green color mask
        '''Hue 25–100 → captures green shades.

            Saturation 30–255 → ensures some color intensity (not gray).

            Value 20–255 → brightness.'''
            
            
        if np.count_nonzero(cleaned) < 200:
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            lower_green = np.array([25, 30, 20])
            upper_green = np.array([100, 255, 255])
            color_mask = cv2.inRange(hsv, lower_green, upper_green)  #White (255) = pixel within green range. Black (0) = not green.
            cleaned = cv2.morphologyEx(color_mask, cv2.MORPH_CLOSE, kernel_close)
            cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel_open)
            
            
    # Step 4: Pick largest contour (biggest object = candidate leaf)
        contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            return None, None, cleaned
        largest = max(contours, key=cv2.contourArea)
        mask = np.zeros_like(gray, dtype=np.uint8)
        cv2.drawContours(mask, [largest], -1, 255, -1)
        return mask, largest, cleaned
    
    #Extract Features
    def extract_features(image, mask, contour):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image_area = image.shape[0] * image.shape[1]

        mask_pixels = np.count_nonzero(mask) if mask is not None else 0
        contour_area = cv2.contourArea(contour) if contour is not None else 0 #geometric measure
        #Helps reject false positives (like full green wall or tiny dot).
        area_ratio = contour_area / image_area if image_area > 0 else 0 # how much frame is covered 
        mask_coverage = mask_pixels / image_area if image_area > 0 else 0 # mostly similar but is pixel basedd

        # edges 
        edges = cv2.Canny(gray, 50, 150) #it detect sharp edges like leaf outliine and veins
        edge_count = np.count_nonzero(edges) #how many edge pixel exist
        edge_density = edge_count / image_area if image_area > 0 else 0 #edges per unit are 

        # skeleton/veins   Leaves mainly show branch-like skeletons. A plain green screen can’t.
        masked_gray = cv2.bitwise_and(gray, gray, mask=mask)
        _, bin_mask = cv2.threshold(masked_gray, 1, 255, cv2.THRESH_BINARY)
        skel = skeletonize(bin_mask // 255)
        skeleton_pixels = np.count_nonzero(skel)
        vein_density = skeleton_pixels / mask_pixels if mask_pixels > 0 else 0

        # intensity variation->Real leaves have shadows, texture, and gradients. but green walls have low variance and entropy
        masked_gray_vals = gray[mask > 0]
        gray_variance = float(np.var(masked_gray_vals)) if masked_gray_vals.size > 0 else 0.0  # how much pixel intensity varied inside the leaaf
        gray_entropy = float(compute_entropy(masked_gray_vals)) if masked_gray_vals.size > 0 else 0.0   #randomness/complexity of pixel intensities

        # color variation
        if mask_pixels > 0:
            masked_pixels = image[mask > 0].reshape(-1, 3)
            color_std = float(np.mean(np.std(masked_pixels, axis=0)))
        else:
            color_std = 0.0

        # solidity
        solidity = 0.0
        if contour is not None:
            hull = cv2.convexHull(contour)  #Convex hull = smallest convex polygon around the shape.
            hull_area = cv2.contourArea(hull)
            if hull_area > 0:
                solidity = contour_area / hull_area

        feats = {
            'image_area': image_area,
            'mask_exists': mask is not None,
            'contour_area': contour_area,
            'mask_pixels': mask_pixels,
            'area_ratio': area_ratio,
            'mask_coverage': mask_coverage,
            'edge_count': edge_count,
            'edge_density': edge_density,
            'vein_density': vein_density,
            'skeleton_pixels': skeleton_pixels,
            'gray_variance': gray_variance,
            'gray_entropy': gray_entropy,
            'color_std': color_std,
            'solidity': solidity
        }
        return feats

    def is_leaf_detector(image, debug=False):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        mask, contour, _ = segment_largest_object(image, gray)

        if mask is None or contour is None:
            return False, {'reason': 'No object detected'}, []

        feats = extract_features(image, mask, contour)

        # heuristics
        votes = {
            'enough_size': feats['contour_area'] > 1000,
            'not_full_frame': feats['area_ratio'] < 0.95,
            'has_edges': feats['edge_count'] > 500,
            'has_veins': feats['vein_density'] > 0.005,
            'has_variation': feats['gray_variance'] > 50,
            'has_entropy': feats['gray_entropy'] > 2.5,
            'color_varied': feats['color_std'] > 5
        }
        score = sum(votes.values())

        # base decision
        is_leaf = votes['has_edges'] and votes['has_entropy'] and votes['enough_size']

        if score < 4:
            is_leaf = False

        # safeguard: flat green wall rejection
        if feats['vein_density'] < 0.001 and feats['solidity'] > 0.9 and feats['gray_entropy'] < 3.0:
            is_leaf = False

        if feats['gray_entropy'] < 2.8 and feats['edge_count'] < 300:
            is_leaf = False

        confidence = float(score) / len(votes)
        if not is_leaf:
            confidence *= 0.5

        failed_reasons = [k for k, v in votes.items() if not v]
        diagnostics = {**feats,        
                    'votes': votes,
                    'score': score,
                    'is_leaf': is_leaf,
                    'failed_reasons': failed_reasons,
                    'confidence': confidence}

        if debug:
            print("Diagnostics: ", diagnostics )
            for key,value in diagnostics.items():
                print(f"{key} :{value}")
            print("Leaf?", is_leaf)
            print(" ----------------------------- ")
        return is_leaf, diagnostics, feats
        
        #votes 
        print("Votes:")
        for vkey, vval in diagnostics['votes'].items():
            print(f"   {vkey}: {vval}")

    if __name__ == "__main__":
        img = cv2.imread("G3.jpg")  # change filename
        is_leaf_flag, diag, feats = is_leaf_detector(img, debug=True)
        print("Final Decision:", "✅ Leaf" if is_leaf_flag else "❌ Not Leaf")
        if not is_leaf_flag:
            print("Reasons:", diag.get('failed_reasons'))

Diagnostics:  {'image_area': 90000, 'mask_exists': True, 'contour_area': 89401.0, 'mask_pixels': 90000, 'area_ratio': 0.9933444444444445, 'mask_coverage': 1.0, 'edge_count': 0, 'edge_density': 0.0, 'vein_density': 3.3333333333333335e-05, 'skeleton_pixels': 3, 'gray_variance': 0.0, 'gray_entropy': 0.0, 'color_std': 0.0, 'solidity': 1.0, 'votes': {'enough_size': True, 'not_full_frame': False, 'has_edges': False, 'has_veins': False, 'has_variation': False, 'has_entropy': False, 'color_varied': False}, 'score': 1, 'is_leaf': False, 'failed_reasons': ['not_full_frame', 'has_edges', 'has_veins', 'has_variation', 'has_entropy', 'color_varied'], 'confidence': 0.07142857142857142}
image_area :90000
mask_exists :True
contour_area :89401.0
mask_pixels :90000
area_ratio :0.9933444444444445
mask_coverage :1.0
edge_count :0
edge_density :0.0
vein_density :3.3333333333333335e-05
skeleton_pixels :3
gray_variance :0.0
gray_entropy :0.0
color_std :0.0
solidity :1.0
votes :{'enough_size': True, 'not_full